
# パラメータの初期化

パラメータにアクセスする方法がわかったので、パラメータを適切に初期化する方法を見てみましょう。 :numref: `sec_numerical_stability`で適切な初期化の必要性について説明しました。深層学習フレームワークは、その層にデフォルトのランダム初期化を提供します。ただし、他のさまざまなプロトコルに従って重みを初期化したい場合もよくあります。このフレームワークは、最も一般的に使用されるプロトコルを提供し、カスタム初期化子を作成することもできます。


In [1]:
import torch
from torch import nn


デフォルトでは、PyTorch は、入力および出力の次元に従って計算された範囲から描画することにより、重み行列とバイアス行列を均一に初期化します。 PyTorch の`nn.init`モジュールは、さまざまなプリセット初期化メソッドを提供します。


In [2]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])


## [**組み込みの初期化**]

組み込みイニシャライザを呼び出すことから始めましょう。以下のコードは、すべての重みパラメータを標準偏差 0.01 のガウス確率変数として初期化し、バイアス パラメータをゼロにクリアします。


In [3]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0089,  0.0039, -0.0204, -0.0059]), tensor(0.))


すべてのパラメータを特定の定数値 (たとえば 1) に初期化することもできます。


In [4]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))


[**特定のブロックに異なる初期化子を適用することもできます。** ] たとえば、以下では、最初の層を Xavier イニシャライザで初期化し、2 番目の層を定数値 42 に初期化します。


In [5]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0542, -0.6922, -0.2629, -0.4876])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])



### 【**カスタム初期化**】

場合によっては、必要な初期化メソッドが深層学習フレームワークによって提供されていないことがあります。以下の例では、次のような奇妙な分布を使用して、重みパラメータ $w$ の初期化子を定義します。

 $$ \begin{aligned} w \sim \begin{cases} U(5, 10) &amp; \text{ 確率 } \frac{1}{4} \ 0 &amp; \text{ 確率 } \frac{1} {2} \ U(-10, -5) &amp; \text{ 確率 } \frac{1}{4} \end{cases} \end{aligned} $$



もう一度、 `net`に適用する`my_init`関数を実装します。


In [6]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-8.4801, -0.0000,  0.0000, -5.6451],
        [ 5.6075,  0.0000,  5.4012, -5.0333]], grad_fn=<SliceBackward0>)


パラメータを直接設定するオプションが常にあることに注意してください。


In [7]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000, -4.6451])


## まとめ

組み込みおよびカスタム初期化子を使用してパラメータを初期化できます。

## 演習

その他の組み込みイニシャライザについては、オンライン ドキュメントを参照してください。



[ディスカッション](https://discuss.d2l.ai/t/8090)
